In [441]:
import import_ipynb
from dijkstra import nodes, arc_matrix, get_shortest_path, dijkstra, format_path

from pprint import pprint
import numpy as np


In [442]:
def init_tamponi_per_reparto():
    tamponi_per_reparto = np.array([0, 0, 0, 3, 0, 0, 0, 6, 0, 0, 0, 0, 12, 0, 0, 0, 7, 0, 0, 0, 10, 0,
        0, 0, 3, 0, 30, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]).astype(int)

    return tamponi_per_reparto

In [443]:
tamponi_per_reparto = init_tamponi_per_reparto()
print(tamponi_per_reparto)

[ 0  0  0  3  0  0  0  6  0  0  0  0 12  0  0  0  7  0  0  0 10  0  0  0
  3  0 30  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]


In [444]:
def estrai_reparti_con_tamponi(tamponi_per_reparto):
    return [i for i,t in enumerate(tamponi_per_reparto) if t > 0]

In [445]:
reparti_con_tamponi = estrai_reparti_con_tamponi(tamponi_per_reparto)

print(reparti_con_tamponi)

for index,node in enumerate(reparti_con_tamponi):
    print(f"{index}. Reparto con indice {node} e nome '{nodes[node]}' ha {tamponi_per_reparto[node]} tamponi")

[3, 7, 12, 16, 20, 24, 26, 28]
0. Reparto con indice 3 e nome '4' ha 3 tamponi
1. Reparto con indice 7 e nome '7' ha 6 tamponi
2. Reparto con indice 12 e nome '11n' ha 12 tamponi
3. Reparto con indice 16 e nome '14' ha 7 tamponi
4. Reparto con indice 20 e nome '19' ha 10 tamponi
5. Reparto con indice 24 e nome '22' ha 3 tamponi
6. Reparto con indice 26 e nome '24' ha 30 tamponi
7. Reparto con indice 28 e nome '25/2' ha 4 tamponi


In [446]:
## SCORE MATRIX

# conteggio dei tamponi in ogni nodo dato un path (oppure un nodo di partenza) 
def compute_tamponi_in_nodo(p, prev, tamponi_per_reparto, tamponi_per_path, start_node=None):
    if prev is None and start_node is not None:
        _, prev = np.array(dijkstra(arc_matrix, start_node))
        tamponi_per_path = -1 * np.ones(len(prev))
    
    path = []
    path.append(p)

    prev = np.array(prev).astype(int)
    while p != -1:
        p = prev[p]
        path.append(p)

    path = path[::-1] # reverse
    path = path[1:] # remove start node

    temp_tamponi = 0
    for p in path:
        if tamponi_per_path[p] == -1:
            # print(p,"aggiungo questo reparto al dizionario")
            temp_tamponi += tamponi_per_reparto[p]
            tamponi_per_path[p] = temp_tamponi
        else:
            temp_tamponi = tamponi_per_path[p]
        
        # print(p, nodes[p], tamponi_per_reparto[p], temp_tamponi)
    
    return tamponi_per_path
        

def compute_score_row(arc_matrix, reparti_con_tamponi, indice_reparto, tamponi_per_reparto):
    time, prev = np.array(dijkstra(arc_matrix, indice_reparto))
    # print(prev)
    
    ## TAMPONI PER PATH
    tamponi_per_path = -1 * np.ones(len(prev))
    for reparto in reparti_con_tamponi:
        tamponi_per_path = compute_tamponi_in_nodo(reparto, prev, tamponi_per_reparto, tamponi_per_path)
    ## TAMPONI IN NODO FINALE
    # tamponi_per_path = tamponi_per_reparto.copy()
    
    
    time = time[reparti_con_tamponi]
    tamponi_per_path = tamponi_per_path[reparti_con_tamponi]
    
    # print(tamponi_per_path)
    density_score = np.divide(tamponi_per_path, time, out=-np.inf*np.ones_like(time), where=time!=0)
    return density_score

# tamponi / minuto
def compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto):
    return compute_density_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto)

def compute_density_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto):
    reparti_con_tamponi = np.array(reparti_con_tamponi)
    score_matrix = np.zeros((len(reparti_con_tamponi), len(reparti_con_tamponi)))
    for i, indice_reparto in enumerate(reparti_con_tamponi):
        score_matrix[i] = compute_score_row(arc_matrix, reparti_con_tamponi, indice_reparto, tamponi_per_reparto)
    return score_matrix

In [447]:
def retrieve_best_score(score_matrix, index=None):
    if index is not None: # per score_matrix
        return np.argmax(score_matrix[index]), np.max(score_matrix[index])
    else:
        return np.argmax(score_matrix), np.max(score_matrix)

In [448]:
score_matrix = compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto)
for index,node in enumerate(reparti_con_tamponi):
    best_index, best_score = retrieve_best_score(score_matrix, index)
    print(f"{index}. Reparto con indice {node} e nome '{nodes[node]}' ha best score in reparto {best_index}. indice {reparti_con_tamponi[best_index]} con nome '{nodes[reparti_con_tamponi[best_index]]}' con score {best_score}")

0. Reparto con indice 3 e nome '4' ha best score in reparto 6. indice 26 con nome '24' con score 2.2941176470588234
1. Reparto con indice 7 e nome '7' ha best score in reparto 6. indice 26 con nome '24' con score 7.2
2. Reparto con indice 12 e nome '11n' ha best score in reparto 6. indice 26 con nome '24' con score 3.2
3. Reparto con indice 16 e nome '14' ha best score in reparto 6. indice 26 con nome '24' con score 2.5294117647058822
4. Reparto con indice 20 e nome '19' ha best score in reparto 6. indice 26 con nome '24' con score 2.5
5. Reparto con indice 24 e nome '22' ha best score in reparto 6. indice 26 con nome '24' con score 3.3
6. Reparto con indice 26 e nome '24' ha best score in reparto 1. indice 7 con nome '7' con score 7.2
7. Reparto con indice 28 e nome '25/2' ha best score in reparto 6. indice 26 con nome '24' con score 1.4166666666666667


In [449]:
def check_reparto_intermedio(start_node, next_index, reparti_con_tamponi, tamponi_per_reparto):
    # controllo: se all'interno del path c'è un nodo con tamponi, allora scelgo quello
    
    path, _ = get_shortest_path(arc_matrix, start_node, reparti_con_tamponi[next_index])
    
    nodo_finale = None
    for p in path[1:-1]:
        if p in reparti_con_tamponi and tamponi_per_reparto[p] > 0:
            nodo_finale = next_index
            next_index = reparti_con_tamponi.index(p)
            break
    
    return next_index, nodo_finale

def get_next_node(start_node, reparti_con_tamponi, score_matrix, tamponi_per_reparto):
    index = reparti_con_tamponi.index(start_node)
    next_index, score = retrieve_best_score(score_matrix, index)
    
    next_index, nodo_finale = check_reparto_intermedio(start_node, next_index, reparti_con_tamponi, tamponi_per_reparto)
    
    score_matrix[:,index] = -np.inf
    return reparti_con_tamponi[next_index], score, nodo_finale

In [450]:
dimensione_piastra = 96 # tamponi
def stop_condition(residual_time, tamponi_trasportati, step):
    return residual_time <= 0 or tamponi_trasportati >= dimensione_piastra or step >= len(reparti_con_tamponi) # aggiungere tempo al lab

In [451]:
def build_tsp_path(arc_matrix, reparti_con_tamponi, tamponi_per_reparto_orig, residual_time, tamponi_trasportati, current_node):
    tamponi_per_reparto = tamponi_per_reparto_orig.copy()
    path = []
    
    # se il nodo di partenza non è un nodo con tamponi, allora trovo il nodo con tamponi con score più alto
    if current_node not in reparti_con_tamponi:
        step = 0
        
        score_row = compute_score_row(arc_matrix, reparti_con_tamponi, current_node, tamponi_per_reparto)
        
        ## TODO: controlla se accorpabile con get_next_node
        next_index, score = retrieve_best_score(score_row)
        next_index, nodo_finale = check_reparto_intermedio(current_node, next_index, reparti_con_tamponi, tamponi_per_reparto)
        next_node = reparti_con_tamponi[next_index]
        ## END TODO
        
        distances, _ = dijkstra(arc_matrix, current_node) # TODO: return prev da usare per compute_tamponi_in_nodo
        tempo_di_percorrenza = distances[next_node]
        residual_time -= tempo_di_percorrenza
        tamponi_per_path = compute_tamponi_in_nodo(next_node, None, tamponi_per_reparto, None, start_node=current_node)
        tamponi_trasportati += tamponi_per_path[next_node]
        
        print(f"{step}. '{nodes[current_node]}' -> '{nodes[next_node]}': {tamponi_per_path[next_node]} tamponi / {tempo_di_percorrenza} minuti = {score} score {(f' (nodo intermedio verso {nodes[reparti_con_tamponi[nodo_finale]]})' if nodo_finale else '')}. Tempo residuo: {residual_time}, Tamponi trasportati: {tamponi_trasportati}.")

        tamponi_per_reparto[next_node] = 0
        
        path.append(next_node)
        current_node = next_node
        step += 1
    
    
    step = 1
    while not stop_condition(residual_time, tamponi_trasportati, step):
        score_matrix = compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto)
        next_node, score, nodo_finale = get_next_node(current_node, reparti_con_tamponi, score_matrix, tamponi_per_reparto)
        
        distances, _ = dijkstra(arc_matrix, current_node) # TODO: return prev da usare per compute_tamponi_in_nodo
        tempo_di_percorrenza = distances[next_node]
        residual_time -= tempo_di_percorrenza
        tamponi_per_path = compute_tamponi_in_nodo(next_node, None, tamponi_per_reparto, None, start_node=current_node)
        tamponi_trasportati += tamponi_per_path[next_node]
        
        print(f"{step}. '{nodes[current_node]}' -> '{nodes[next_node]}': {tamponi_per_path[next_node]} tamponi / {tempo_di_percorrenza} minuti = {score} score {(f' (nodo intermedio verso {nodes[reparti_con_tamponi[nodo_finale]]})' if nodo_finale else '')}. Tempo residuo: {residual_time}, Tamponi trasportati: {tamponi_trasportati}.")
        
        tamponi_per_reparto[next_node] = 0
        
        path.append(next_node)
        current_node = next_node
        step += 1
    
    return path

In [452]:
start_node = nodes.index('4')
residual_time = 120
tamponi_trasportati = 0

path = build_tsp_path(arc_matrix, reparti_con_tamponi, tamponi_per_reparto, residual_time, tamponi_trasportati, start_node)
print(f"Path: {format_path(path)}")

1. '4' -> '7': 9.0 tamponi / 12.0 minuti = 2.2941176470588234 score  (nodo intermedio verso 24). Tempo residuo: 108.0, Tamponi trasportati: 9.0.
2. '7' -> '24': 30.0 tamponi / 5.0 minuti = 6.0 score . Tempo residuo: 103.0, Tamponi trasportati: 39.0.
3. '24' -> '11n': 12.0 tamponi / 15.0 minuti = 0.8 score . Tempo residuo: 88.0, Tamponi trasportati: 51.0.
4. '11n' -> '14': 7.0 tamponi / 10.0 minuti = 0.7 score . Tempo residuo: 78.0, Tamponi trasportati: 58.0.
5. '14' -> '19': 10.0 tamponi / 31.0 minuti = 0.3225806451612903 score . Tempo residuo: 47.0, Tamponi trasportati: 68.0.
6. '19' -> '22': 3.0 tamponi / 10.0 minuti = 0.3 score . Tempo residuo: 37.0, Tamponi trasportati: 71.0.
7. '22' -> '25/2': 4.0 tamponi / 18.0 minuti = 0.2222222222222222 score . Tempo residuo: 19.0, Tamponi trasportati: 75.0.
Path: 7 -> 24 -> 11n -> 14 -> 19 -> 22 -> 25/2


In [453]:
# TODO
# matrice di score tra start_node e i reparti con tamponi: sembra OK
# matrice di score tra i reparti con tamponi e laboratorio + aggiungere nelle stop condition
# aggiungere tempo di percorrenza (corridoio) A -> B

start_node = nodes.index('X2')
residual_time = 120
tamponi_trasportati = 0
path = build_tsp_path(arc_matrix, reparti_con_tamponi, tamponi_per_reparto, residual_time, tamponi_trasportati, start_node)
print(f"Path: {format_path(path)}")

0. 'X2' -> '7': 6.0 tamponi / 4.0 minuti = 4.0 score  (nodo intermedio verso 24). Tempo residuo: 116.0, Tamponi trasportati: 6.0.
1. '7' -> '24': 30.0 tamponi / 5.0 minuti = 6.0 score . Tempo residuo: 111.0, Tamponi trasportati: 36.0.
2. '24' -> '11n': 12.0 tamponi / 15.0 minuti = 0.8 score . Tempo residuo: 96.0, Tamponi trasportati: 48.0.
3. '11n' -> '14': 7.0 tamponi / 10.0 minuti = 0.7 score . Tempo residuo: 86.0, Tamponi trasportati: 55.0.
4. '14' -> '19': 10.0 tamponi / 31.0 minuti = 0.3225806451612903 score . Tempo residuo: 55.0, Tamponi trasportati: 65.0.
5. '19' -> '22': 3.0 tamponi / 10.0 minuti = 0.3 score . Tempo residuo: 45.0, Tamponi trasportati: 68.0.
6. '22' -> '25/2': 4.0 tamponi / 18.0 minuti = 0.2222222222222222 score . Tempo residuo: 27.0, Tamponi trasportati: 72.0.
7. '25/2' -> '4': 3.0 tamponi / 39.0 minuti = 0.07692307692307693 score . Tempo residuo: -12.0, Tamponi trasportati: 75.0.
Path: 7 -> 24 -> 11n -> 14 -> 19 -> 22 -> 25/2 -> 4
